In [14]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np
from datetime import datetime
import scipy.stats

In [15]:
import warnings
warnings.filterwarnings("ignore")

### Read in Processed Tweets

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
tweets = pd.read_csv('/content/drive/My Drive/tweet_output_predictions.csv')

In [18]:
tweets = tweets[["Date", "Location", "Labels"]]

In [19]:
tweets = tweets[tweets["Location"].str.contains(", United States")]

In [20]:
tweets["State"] = tweets["Location"].apply(lambda x: x.split(",")[0])

In [21]:
tweets.Labels.unique()

array(['Neutral', 'Irrelevant', 'Positive', 'Negative'], dtype=object)

In [22]:
tweets.State = tweets.State.apply(lambda x: x.strip())

In [23]:
tweets = tweets[tweets.Labels != "Irrelevant"]

In [24]:
conditions = [
    tweets['Labels'] == "Positive",
    tweets['Labels'] == "Neutral",
    tweets["Labels"] == "Negative"
]

choices = [1,0, -1]

tweets['Labels'] = np.select(conditions, choices, default=0)

In [25]:
tweets.Date = tweets.Date.apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S+00:00').strftime('%Y'))

In [26]:
tweets["Count"] = 1

In [143]:
tweets_grouped = tweets.groupby(["Date", "State"]).agg({'Labels':'mean', 'Count':'sum'}).reset_index()
tweets_grouped

,Date,State,Labels,Count
0,2020,Alabama,0.078947,76
1,2020,Alaska,0.240000,25
2,2020,Arenal,-1.000000,1
3,2020,Arizona,0.093385,257
4,2020,Arkansas,0.190476,21
...,...,...,...,...
172,2022,Virginia,0.254386,114
173,2022,Washington,0.559454,513
174,2022,West Virginia,0.750000,4
175,2022,Wisconsin,0.476190,21


In [28]:
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/List_of_United_States_presidential_election_results_by_state"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

In [29]:
soup = BeautifulSoup(response.text, 'html.parser')
states=soup.find('table',{'class':"wikitable"})

In [30]:
state_df=pd.read_html(str(states)) #into list
state_df=pd.DataFrame(state_df[0]) #into dataframe

In [31]:
state_df.columns = state_df.iloc[0]

In [32]:
state_df = state_df.iloc[1:] #drop the first row since we already established it as the header

In [33]:
state_df = state_df[["State", "2016 ‡", "2020"]] #only get the last two presidential elections

In [34]:
state_df = state_df.loc[:,~state_df.columns.duplicated(keep = 'first')]

In [35]:
state_df = state_df.dropna()
state_df = state_df[state_df.State != 'State']

In [37]:
state_df.rename(columns = {"2016 ‡":"2016"}, inplace = True)

#### Merge with tweet sentiment

In [89]:
tweets_grouped_president = tweets_grouped.copy()

In [90]:
conditions = [
    tweets_grouped['Date'] == "2020",
    tweets_grouped['Date'] != "2020"
]

choices = ["Previous election", "Current election"]

tweets_grouped_president['Date'] = np.select(conditions, choices, default=0)

In [91]:
conditions = [
    tweets_grouped['Date'] == "2020",
    tweets_grouped['Date'] != "2020"
]

choices = ["Previous election", "Current election"]

tweets_grouped_president['Date'] = np.select(conditions, choices, default=0)

In [92]:
tweets_grouped_president["political_affiliation"] = None

In [93]:
for i in range(len(tweets_grouped_president)):
    if tweets_grouped_president["Date"][i] == "Previous election":
        if tweets_grouped_president["State"][i] in rstate_2016_pres:
            tweets_grouped_president["political_affiliation"][i] = "R"
        else:
            tweets_grouped_president["political_affiliation"][i] = "D"
    else:
        if tweets_grouped_president["State"][i] in rstate_2020_pres:
            tweets_grouped_president["political_affiliation"][i] = "R"
        else:
            tweets_grouped_president["political_affiliation"][i] = "D"

In [94]:
tweets_grouped_president_all = tweets_grouped_president.copy()

In [95]:
tweets_grouped_president = tweets_grouped_president.groupby(["Date", "political_affiliation"]).agg({'Labels':'mean', 'Count':'sum'}).reset_index()

In [96]:
order_list = ['Previous election', 'Current election']

In [126]:
tweets_grouped_state = tweets_grouped.copy()

In [127]:
tweets_grouped_state = tweets_grouped_state[["State", "Labels"]].groupby("State").mean().reset_index()

In [128]:
governor_sentiment = governors.merge(tweets_grouped_state, on = "State", how = "left")

In [129]:
governor_sentiment.dropna(inplace = True)

In [131]:
governor_sentiment

,State,Party.1,Labels
0,Alabama,Republican,0.299784
1,Alaska,Republican,0.487743
2,Arizona,Republican,0.172021
3,Arkansas,Republican,0.253728
4,California,Democratic,0.370379
5,Colorado,Democratic,0.365240
7,Delaware,Democratic,0.408120
8,Florida,Republican,0.173856
9,Georgia,Republican,0.300910
10,Hawaii,Democratic,0.250242


In [71]:
# https://ballotpedia.org/Election_results,_2021:_Party_control_of_state_legislatures

wikiurl="https://ballotpedia.org/Election_results,_2021:_Party_control_of_state_legislatures"
table_class="marqueetable sortable jquery-tablesorter"
response=requests.get(wikiurl)

In [72]:
soup = BeautifulSoup(response.text, 'html.parser')

In [73]:
state_legislature =soup.find('table',{'class':"marqueetable"})

In [74]:
state_legislature=pd.read_html(str(state_legislature)) #into list
state_legislature=pd.DataFrame(state_legislature[0]) #into dataframe

In [75]:
state_legislature.columns = state_legislature.columns.droplevel(0).droplevel(0)
state_legislature = state_legislature[["State", "State Senate", "State House"]]

In [76]:
state_legislature = state_legislature.loc[:,~state_legislature.columns.duplicated(keep = 'last')]

In [78]:
state_senate = state_legislature[["State", "State Senate"]]
state_house = state_legislature[["State", "State House"]]

Misalignment 

In [135]:
new = state_legislature.merge(governors, on="State")

In [137]:
misalignment = []
for index, row in new.iterrows():
  if row['State Senate'] == row['State House'] and row['State Senate'] == row['Party.1']:
    misalignment.append(0)
  else:
    misalignment.append(1)

In [138]:
new["misalignment"] = misalignment
new


,State,State Senate,State House,Party.1,misalignment
0,Alabama,Republican,Republican,Republican,0
1,Alaska,Republican,Split,Republican,1
2,Arizona,Republican,Republican,Republican,0
3,Arkansas,Republican,Republican,Republican,0
4,California,Democratic,Democratic,Democratic,0
5,Colorado,Democratic,Democratic,Democratic,0
6,Connecticut,Democratic,Democratic,Democratic,0
7,Delaware,Democratic,Democratic,Democratic,0
8,Florida,Republican,Republican,Republican,0
9,Georgia,Republican,Republican,Republican,0


In [139]:
c = 0
li = []
for index, row in new.iterrows():
  if row["misalignment"] == 1:
    c = c +1
print(c)

13


In [142]:
select_color = new.loc[new['misalignment'] == 1]
select_color.reset_index()

new_df = select_color[['State', 'State Senate', 'State House', 'Party.1', 'misalignment']]
new_df

,State,State Senate,State House,Party.1,misalignment
1,Alaska,Republican,Split,Republican,1
15,Kansas,Republican,Republican,Democratic,1
16,Kentucky,Republican,Republican,Democratic,1
17,Louisiana,Republican,Republican,Democratic,1
19,Maryland,Democratic,Democratic,Republican,1
20,Massachusetts,Democratic,Democratic,Republican,1
21,Michigan,Republican,Republican,Democratic,1
22,Minnesota,Republican,Democratic,Democratic,1
32,North Carolina,Republican,Republican,Democratic,1
37,Pennsylvania,Republican,Republican,Democratic,1


In [144]:
tweets_grouped_mis = tweets_grouped.copy()

In [146]:
tweets_grouped_mis = tweets_grouped_mis[["State", "Labels"]].groupby("State").mean().reset_index()

In [149]:
mis_sentiment = new_df.merge(tweets_grouped_state, on = "State", how = "left")
mis_sentiment

,State,State Senate,State House,Party.1,misalignment,Labels
0,Alaska,Republican,Split,Republican,1,0.487743
1,Kansas,Republican,Republican,Democratic,1,0.394152
2,Kentucky,Republican,Republican,Democratic,1,0.393739
3,Louisiana,Republican,Republican,Democratic,1,0.236652
4,Maryland,Democratic,Democratic,Republican,1,0.552047
5,Massachusetts,Democratic,Democratic,Republican,1,0.430925
6,Michigan,Republican,Republican,Democratic,1,0.310848
7,Minnesota,Republican,Democratic,Democratic,1,NaN
8,North Carolina,Republican,Republican,Democratic,1,0.320950
9,Pennsylvania,Republican,Republican,Democratic,1,0.328264


In [150]:
mis_sentiment.dropna(inplace = True)

In [151]:
mis_r = mis_sentiment[mis_sentiment["Party.1"] == "Republican"]["Labels"]
mis_d = mis_sentiment[mis_sentiment["Party.1"] == "Democratic"]["Labels"]

In [152]:
scipy.stats.ttest_ind(mis_r, mis_d)

Ttest_indResult(statistic=2.4380677508314927, pvalue=0.03748324938621509)

In [153]:
mis_r1 = mis_sentiment[mis_sentiment["State Senate"] == "Republican"]["Labels"]
mis_d1 = mis_sentiment[mis_sentiment["State Senate"] == "Democratic"]["Labels"]

In [155]:
scipy.stats.ttest_ind(mis_r1, mis_d1)

Ttest_indResult(statistic=-1.357433205165445, pvalue=0.2077025968259517)

In [156]:
mis_r2 = mis_sentiment[mis_sentiment["State House"] == "Republican"]["Labels"]
mis_d2 = mis_sentiment[mis_sentiment["State House"] == "Democratic"]["Labels"]

In [157]:
scipy.stats.ttest_ind(mis_r2, mis_d2)

Ttest_indResult(statistic=-3.562018655583104, pvalue=0.007378977702753433)